In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
# from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re
from nltk.stem import WordNetLemmatizer


In [2]:
df_hash = {
#     "cooking": pd.read_csv("cooking.csv"),
#     "crypto": pd.read_csv("crypto.csv"),
    "robotics": pd.read_csv("data/robotics.csv"),
#     "biology": pd.read_csv("biology.csv"),
#     "travel": pd.read_csv("travel.csv"),
#     "diy": pd.read_csv("diy.csv"),
}

print(df_hash["robotics"].iloc[1])

id                                                         2
title      How can I modify a low cost hobby servo to run...
content    <p>I've got some hobby servos (<a href="http:/...
tags                                         control rcservo
Name: 1, dtype: object


In [3]:
df_hash['robotics'].columns.values

array(['id', 'title', 'content', 'tags'], dtype=object)

In [4]:
# biologyTable.shape
print(df_hash["robotics"].shape)
print df_hash["robotics"].tags.head(5)

(2771, 4)
0                          soccer control
1                         control rcservo
2                               gait walk
3    microcontroller arduino raspberry-pi
4                     motion-planning rrt
Name: tags, dtype: object


In [5]:
import nltk
# nltk.download()
from nltk.corpus import stopwords
# In Python, searching a set is much faster than searching a list, so convert the stop words to a set
stops = set(stopwords.words("english"))  

https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words

In [6]:
def separate_tags(tags):
#     print tags
    letters_only = re.sub("[^a-zA-Z]", " ", tags) 
#     return map( lambda x: [x], letters_only.lower().split() )
    return letters_only.lower().split()

# print separate_tags("hello googbye13 42 HI")

In [7]:
def strip_html(raw_html):
    if raw_html:
        # remove html tags
        soup = BeautifulSoup(raw_html, "html.parser")

        [s.extract() for s in soup(['pre', 'code'])]
            
        question_text = soup.get_text()
        
        # remove everything but letters
        letters_only = re.sub("[^a-zA-Z]", " ", question_text) 
        
            
        # normalize case
        words = letters_only.lower().split()   

        # remove stopwords         
        meaningful_words = [w for w in words if not w in stops] 
        
        wordnet_lemmatizer = WordNetLemmatizer()
        meaningful_word_stems = map(lambda x: wordnet_lemmatizer.lemmatize(x) , meaningful_words)
        return( " ".join( meaningful_word_stems )) 
    else:
        return ""
    
# strip_html(df_hash['robotics'].content[0])

In [8]:
print "Cleaning and parsing the training set of tags for each topic...\n"

for topic, df in df_hash.iteritems():
    print topic
    print "'s size is "
    
    df.content = df.content.apply(lambda x: strip_html(x) )
    df.tags = df.tags.apply(lambda y: separate_tags(y) )
    df.tags.replace(['', ' '], np.nan, inplace=True)
    df.dropna(subset=['tags'], inplace=True)
    print df.tags
    print df.content.head(4)

Cleaning and parsing the training set of tags for each topic...

robotics
's size is 
0                                       [soccer, control]
1                                      [control, rcservo]
2                                            [gait, walk]
3               [microcontroller, arduino, raspberry, pi]
4                                 [motion, planning, rrt]
5                                    [software, platform]
6                                     [software, circuit]
7                [odometry, localization, kalman, filter]
8                                              [untagged]
9                                     [soccer, mechanism]
10                     [computer, vision, wheeled, robot]
11                                           [quadcopter]
12                                               [servos]
13                          [localization, mobile, robot]
14                                        [kinect, input]
15                                          

In [9]:
print df_hash["robotics"].shape


(2771, 4)


In [10]:
# from sklearn.base import BaseEstimator, TransformerMixin

# class QuestionStripper(BaseEstimator, TransformerMixin):

#     def __init__(self):
#         self = self
#         self.df = df  # e.g. pass in a column name to extract
#         self.topic_size = df.size
        
#     def transform(self, X, y=None):
#         return do_something_to(X, self.vars)  # where the actual feature extraction happens
#         print "'s size is "
#         print self.topic_size
#         return X.apply(lambda x: strip_html(x) )
        
    
#     def fit(self, X, y=None):
#         return self  # generally does nothing

In [11]:
print df_hash["robotics"].tags.head(5)
print df_hash["robotics"].shape
print df.content.head(4)

0                            [soccer, control]
1                           [control, rcservo]
2                                 [gait, walk]
3    [microcontroller, arduino, raspberry, pi]
4                      [motion, planning, rrt]
Name: tags, dtype: object
(2771, 4)
0    imagine programming wheel soccer robot type co...
1    got hobby servo power hd mg like able control ...
2    http www oricomtech com project leg time htm l...
3    looking starting point project preferably usin...
Name: content, dtype: object


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, f1_score, make_scorer, accuracy_score, confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier, OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from math import sqrt
from skmultilearn.ensemble import RakelD, RakelO
from skmultilearn.problem_transform import BinaryRelevance, LabelPowerset
from sklearn.naive_bayes import MultinomialNB

# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import SelectFromModel
from time import time
from skmultilearn.problem_transform import BinaryRelevance  
# from nltk.classify.scikitlearn import SklearnClassifier
# from collections import defaultdict
# from sklearn.linear_model import LogisticRegressionCV

# from sklearn.cluster import MiniBatchKMeans
# parameters = {
#     'rakel__labelset_size': range(1, 3),
#     'classifier': [LabelPowerset(), SVC()],
#     'rakel__classifier': "classifier"
#     }

# parameters = {'tree__max_depth': range(7, 20),
#               "tree__n_estimators": [10, 50, 100],
#               "tree__bootstrap": [ False],
#               "tree__criterion": ["gini", "entropy"]}
# parameters = {
#     'rakel__labelset_size': range(1, 3),
#     'vectorizer__ngram_range': range(1,2),
#     'rakel__classifier': [LabelPowerset(), BinaryRelevance()],
#     'rakel__classifier__classifier': [MultinomialNB()],
#     'rakel__classifier__classifier__alpha': [0.7, 1.0],
# }

# TfidfTransformer = TfidfVectorizer(analyzer = "word",   \
#                          tokenizer = None,    \
#                          preprocessor = None, \
#                          stop_words = None,   \
#                          max_features = 5000) 
        

# use a full grid over all parameters
# param_grid = {"max_depth": [3, None],
#               "max_features": [1, 3, 10],





# Set the parameters by cross-validation
# parameters = {'svc__estimator__kernel': ['rbf', 'poly']}
#               'svc__estimator__degree':[1, 2, 3] }

# parameters = dict(svc__estimator__C=np.logspace(-5,0,5), svc__estimator__gamma=np.logspace(-2, 2, 10))

#     print("Best parameters set found on development set:")
#     print()
#     print(clf.best_params_)
#     print()
#     print("Grid scores on development set:")

        
  
for topic, df in df_hash.iteritems():
    print "splitting"
    
    
#     tf = TfidfVectorizer()
#     X = tf.fit_transform(df.content.values)
#     Y = df.tags.values
    print "len df.tags"
    print len(df.tags)
    print df.tags.shape
    print df.tags.head(5)
#     lab_en = LabelEncoder()
    mlb = MultiLabelBinarizer()
    Y = pd.DataFrame( mlb.fit_transform(df.tags) )
#     print df.tags.head(5)
#     d = defaultdict(LabelEncoder)
#     df.apply(LabelEncoder().fit_transform)
#     Y = df.tags.apply(lambda x: tuple(x)).apply(lambda x: d[x].fit_transform(x)).apply(lambda x: np.asarray(x) )

#     print Y
#     print "len Y"
#     print lenY)
    print "y shape"
    print Y.shape
    print "y"
    print Y.head(5)
    print type(Y)
#     true_k =  np.unique(Y).shape[0]
    true_k = len(Y)
    print "true K: \n"
    print int(sqrt(true_k))
    
    
    # steps = [('strip_questions', QuestionStripper()),
#     steps = [   ('vectorizer', CountVectorizer(stop_words="english", strip_accents='unicode')),
    steps = [('tfidf', TfidfVectorizer(tokenizer=lambda doc: doc, lowercase=False , analyzer = "word", max_df=0.5, sublinear_tf=True, stop_words="english",max_features = 5000 ) ),
#             ('rakel', RakelD(BinaryRelevance(MultinomialNB())))]
            ('rf_classifier', RandomForestClassifier( n_estimators=int(sqrt(true_k)), random_state=42, max_depth=13, ))]
#                 ("clf", LogisticRegressionCV(Cs=range(1,10), solver='sag', max_iter=100, random_state=42, multi_class="ovr",cv=10) )]
#             ("dt_class", DecisionTreeClassifier(random_state = 42, max_features = "auto", class_weight = "auto")),
#             ("ada_boost", AdaBoostClassifier())]
#             ( 'kmeans_cluster', MiniBatchKMeans(n_clusters=int(sqrt(true_k)), init='k-means++', n_init=1,
#                              init_size=1000, batch_size=1000) ) ]
    pipeline = Pipeline(steps)
#     Y = mlb.fit_transform(df.tags)z
#     print Y.shape
#     print Y
#     print df.tags.head(5)
#     X_train, X_test, y_train, y_test_vect = train_test_split(df.content,\
#                                                         Y, \
#                                                         test_size=0.33,\
#                                                         random_state=42)
    


#     scorer = make_scorer(f1_score, average="weighted")
#     Y_test = mlb.fit_transform(y_test)
#     scores = ['precision', 'recall']

#     for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     cv = GridSearchCV( pipeline, scoring='f1')
#     cv = GridSearchCV( pipeline, param_grid = parameters, scoring='f1')
#                            scoring='%s_macro' % score)

#     kfold = KFold(n_splits=10, random_state=seed)
#     X = df.content
#     print X.shape
#     skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
#     mse = []
#     fold_count = 0
#     for train_index, test_index in skf.split(X, Y):
#         print("Processing fold %s" % fold_count)
#         train_fold = X.ix[train_index]
#         test_fold = X.ix[test_index]
        
#         # Get training examples
#         X_train = train_fold.content.values
#         y_train = train_fold.tags

#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = Y[train_index], Y[test_index]
    t0 = time()
#     clf = GridSearchCV(pipeline, parameters, cv=5)
#     clf.fit( X, Y )
#     pipeline.fit(X_train, y_train)
#     y_predicted_vect = pipeline.predict(X_test)
#     print("done in %0.3fs" % (time() - t0))

#     print ("Max depth: %0.3f" % clf.max_depth_ )
#     print("Best score: %0.3f" % clf.best_score_)
#     print("Best parameters set:")
#     best_parameters = clf.best_estimator_.get_params()
#     for param_name in sorted(parameters.keys()):
#         print("\t%s: %r" % (param_name, best_parameters[param_name]))

#     results = cross_val_score(clf, X, Y)
#     print results
#     y_prediction = clf.predict( X_test )
    y_prediction =  cross_val_predict(pipeline, df.tags, Y, cv=5, )
    print("done in %0.3fs" % (time() - t0))
    print "y_prediction: \n"
    print y_prediction.shape
#     print y_prediction
    print type(y_prediction)
    print type(y_prediction[0])
    print y_prediction.astype(int)
#     print y_predicted_vect.to_dense
    
    Y_inv = mlb.inverse_transform(y_prediction)
#     y_test_inv = mlb.inverse_transform(y_test_vect.values)
#     y_pred_inv = mlb.inverse_transform(y_predicted_vect)
#     y_pred_inv = y_predicted_vect.apply(lambda x: d[x].inverse_transform(x))
#     y_test_inv = y_test_vect.apply(lambda x: d[x].inverse_transform(x))

#     print clf.best_estimator_
    print "baallss"
#     print Y_inv
    print accuracy_score(Y, y_prediction) 
    report = classification_report(Y, y_prediction) 
    print(report)
    
#     print cv.best_score_
#     print cv.best_params_
#     print clf.feature_importances_  
#     model = SelectFromModel(clf, prefit=True)
#     print model





splitting
len df.tags
2771
(2771,)
0                            [soccer, control]
1                           [control, rcservo]
2                                 [gait, walk]
3    [microcontroller, arduino, raspberry, pi]
4                      [motion, planning, rrt]
Name: tags, dtype: object
y shape
(2771, 266)
y
   0    1    2    3    4    5    6    7    8    9   ...   256  257  258  259  \
0    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
1    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    1   
3    0    0    0    0    0    0    0    0    1    0 ...     0    0    0    0   
4    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   

   260  261  262  263  264  265  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0

/Users/vlevitas-local/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# y = mlb.inverse_transform(Y)
index = 0
for actual, pred in zip(df.tags, Y_inv):
        print('index: {0} :: {1} => {2}'.format(index, actual, ', '.join(pred)))
        index = index +1

        

index: 0 :: ['soccer', 'control'] => control
index: 1 :: ['control', 'rcservo'] => control
index: 2 :: ['gait', 'walk'] => 
index: 3 :: ['microcontroller', 'arduino', 'raspberry', 'pi'] => arduino, pi, raspberry
index: 4 :: ['motion', 'planning', 'rrt'] => motion
index: 5 :: ['software', 'platform'] => 
index: 6 :: ['software', 'circuit'] => 
index: 7 :: ['odometry', 'localization', 'kalman', 'filter'] => filter
index: 8 :: ['untagged'] => 
index: 9 :: ['soccer', 'mechanism'] => 
index: 10 :: ['computer', 'vision', 'wheeled', 'robot'] => robot
index: 11 :: ['quadcopter'] => 
index: 12 :: ['servos'] => 
index: 13 :: ['localization', 'mobile', 'robot'] => mobile, robot
index: 14 :: ['kinect', 'input'] => 
index: 15 :: ['wheel'] => 
index: 16 :: ['control', 'gyroscope', 'balance', 'two', 'wheeled'] => control
index: 17 :: ['design', 'underwater', 'auv'] => 
index: 18 :: ['underwater', 'battery', 'auv'] => 
index: 19 :: ['electronics', 'protection'] => 
index: 20 :: ['localization', 'gps']

In [17]:
print y_prediction.shape
# print y_test[0]

(2771, 266)


In [ ]:
# print confusion_matrix(Y, y_prediction) 
# print y_pred_inv
# print mlb.inverse_transform(0)


https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/

In [ ]:
# Numpy arrays are easy to work with, so convert the result to an 
# array
# train_data_features = train_data_features.toarray()
# report = classification_report(Y, y_prediction) 
# print report
f1_score(Y, y_prediction, average='weighted')

In [ ]:
# print train_data_features.shape
# print "4"
